In [1]:
"""Collection of NASNet models
The reference paper:
 - [Learning Transferable Architectures for Scalable Image Recognition]
    (https://arxiv.org/abs/1707.07012)
The reference implementation:
1. TF Slim
 - https://github.com/tensorflow/models/blob/master/research/slim/nets/
   nasnet/nasnet.py
2. TensorNets
 - https://github.com/taehoonlee/tensornets/blob/master/tensornets/nasnets.py
"""
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

import warnings

import nasnet_code as nasnet

import keras
import numpy as np
import keras.backend as K
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import Conv2D
from keras.layers import SeparableConv2D
from keras.layers import concatenate
from keras.layers import add
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.applications.inception_v3 import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras import backend as K

_BN_DECAY = 0.9997
_BN_EPSILON = 1e-3

Using TensorFlow backend.


In [2]:
"""Retrieve the CIFAR-10 dataset and process the data."""

nb_classes = 10
batch_size = 64
input_dim = 32*32*3
nb_train = 50000
nb_test = 10000
# Get the data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# Preprocess it
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

print("Cifar 100 data retrieved and processed")

Cifar 100 data retrieved and processed


In [3]:
# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [4]:
act = "relu"

In [5]:
# data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=True,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)



In [6]:
input_shape=x_train.shape[1:]

model = nasnet.NASNetCIFAR(act=act,input_shape=input_shape, dropout=0.2)
model.compile(loss='categorical_crossentropy',
        optimizer="adam",
        metrics=['accuracy'])

C:\Users\Eric\Desktop\Programming\PYTHON\AI\TENSORFLOW\KERAS\learning_keras\ALCAIDE ACTIVATION\development\NASNET\nasnet_code.py:253: UserWarning: Weights of NASNet models have not been ported yet for Keras.
  warnings.warn('Weights of NASNet models have not been ported yet for Keras.')


In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 15, 15, 96)   2592        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 15, 15, 96)   384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 15, 15, 96)   0           stem_bn1[0][0]                   
__________________________________________________________________________________________________
reduction_

normal_add_1_16 (Add)           (None, 1, 1, 128)    0           separable_conv_2_bn_normal_left1_
                                                                 normal_bn_1_16[0][0]             
__________________________________________________________________________________________________
normal_concat_16 (Concatenate)  (None, 1, 1, 768)    0           adjust_bn_16[0][0]               
                                                                 normal_add_2_16[0][0]            
                                                                 normal_add_5_16[0][0]            
                                                                 normal_add_3_16[0][0]            
                                                                 normal_add_4_16[0][0]            
                                                                 normal_add_1_16[0][0]            
__________________________________________________________________________________________________
activation

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=128,  verbose=1, shuffle=True, validation_split=0.1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 328s 7ms/step - loss: 1.9296 - acc: 0.3007 - val_loss: 4.0314 - val_acc: 0.1248
Epoch 2/10
45000/45000 [==============================] - 264s 6ms/step - loss: 1.5863 - acc: 0.4249 - val_loss: 2.1392 - val_acc: 0.1986
Epoch 3/10
 5120/45000 [==>...........................] - ETA: 3:47 - loss: 1.4194 - acc: 0.4891

In [8]:
epochs = 10
batch_size = 128
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))

Epoch 1/10
390/390 [==============================] - 434s 1s/step - loss: 2.2876 - acc: 0.1687 - val_loss: 2.3534 - val_acc: 0.1343
Epoch 2/10
390/390 [==============================] - 356s 913ms/step - loss: 2.0770 - acc: 0.2366 - val_loss: 3.0357 - val_acc: 0.1144
Epoch 3/10
390/390 [==============================] - 355s 911ms/step - loss: 2.0098 - acc: 0.2630 - val_loss: 5.3526 - val_acc: 0.1361
Epoch 4/10
390/390 [==============================] - 354s 909ms/step - loss: 2.0221 - acc: 0.2516 - val_loss: 5.4078 - val_acc: 0.2082
Epoch 5/10
390/390 [==============================] - 354s 908ms/step - loss: 1.8747 - acc: 0.2905 - val_loss: 4.7676 - val_acc: 0.2087
Epoch 6/10
390/390 [==============================] - 354s 909ms/step - loss: 1.8366 - acc: 0.3063 - val_loss: 3.0958 - val_acc: 0.2362
Epoch 7/10
390/390 [==============================] - 358s 919ms/step - loss: 1.8187 - acc: 0.3032 - val_loss: 7.5510 - val_acc: 0.1865
Epoch 8/10
390/390 [==============================]

In [9]:
print(his.history)

{'val_loss': [2.3534206672668456, 3.0357324539184569, 5.3526146324157713, 5.4077903297424319, 4.7675523956298829, 3.095777855682373, 7.5509709411621095, 4.0381895378112791, 4.3660222114562988, 4.714599859619141], 'val_acc': [0.1343, 0.1144, 0.1361, 0.2082, 0.2087, 0.23619999999999999, 0.1865, 0.24340000000000001, 0.2228, 0.21870000000000001], 'loss': [2.2874600249373995, 2.0768414797368324, 2.0094536498910847, 2.0221502793152668, 1.8746323556061697, 1.8366742477915705, 1.8186790244029354, 1.711026313865571, 1.6304989059192461, 1.5626557861569221], 'acc': [0.16875200512357963, 0.23656560796592951, 0.26313362206294438, 0.2516642605164589, 0.29062399743821021, 0.30630413858524147, 0.30319618223606104, 0.34454202759063202, 0.38376243180635078, 0.41973452035303027]}
